In [3]:
import os
import sys
import time
import requests
import numpy as np
import pandas as pd

In [12]:
# input csv with IDs
df = pd.read_csv("~/metagenomic_deepfri/ted_one_for_one.csv")
id_column_name="ID"

In [10]:
# output dir
pdb_dir="ted_structures"
if not os.path.exists(pdb_dir):
    os.makedirs(pdb_dir)

In [15]:
# ---------------- Config ----------------
AF_URL = "https://alphafold.ebi.ac.uk/files/AF-{acc}-F1-model_v6.pdb"
TIMEOUT = 30
RETRIES = 3
BACKOFF = 2.0  # seconds

PLDDT_KEEP_MIN = 70.0      # delete files with mean pLDDT below this
DELETE_LOW_CONF = True      # set False if you only want to flag, not delete
SUMMARY_CSV = "ted_structures_download_summary.csv"
# ---------------------------------------

# Ensure the output directory exists
if not os.path.exists(pdb_dir):
    os.makedirs(pdb_dir)

def warn(msg: str):
    print(f"[WARN] {msg}", file=sys.stderr)

def info(msg: str):
    print(f"[INFO] {msg}")

def download_pdb(accession: str, output_dir: str) -> str | None:
    """Download AlphaFold PDB; returns local filepath or None on failure."""
    url = AF_URL.format(acc=accession)
    filepath = os.path.join(output_dir, f"{accession}.pdb")

    session = requests.Session()
    headers = {
        "User-Agent": "afdb-downloader/1.0 (bioinfo script)",
        "Accept": "chemical/x-pdb, text/plain, */*",
    }

    for attempt in range(1, RETRIES + 1):
        try:
            resp = session.get(url, headers=headers, timeout=TIMEOUT)
            if resp.status_code == 404:
                warn(f"{accession}: AFDB entry not found (HTTP 404).")
                return None
            resp.raise_for_status()

            with open(filepath, "wb") as f:
                f.write(resp.content)

            if b"ATOM" not in resp.content:
                warn(f"{accession}: downloaded file lacks ATOM records; file may be invalid/corrupt.")
            info(f"Downloaded {accession}.pdb")
            return filepath

        except requests.exceptions.RequestException as e:
            if attempt < RETRIES:
                warn(f"{accession}: download error ({e}); retry {attempt}/{RETRIES-1} after {BACKOFF}s.")
                time.sleep(BACKOFF)
            else:
                warn(f"{accession}: download failed after {RETRIES} attempts. Error: {e}")
                return None

def plddt_stats_from_pdb(filepath: str) -> dict | None:
    """
    Parse pLDDT values from PDB (AlphaFold stores pLDDT in B-factor).
    Returns dict with stats or None if no values found.
    """
    plddts = []
    try:
        with open(filepath, "rt", encoding="utf-8", errors="replace") as fh:
            for line in fh:
                # PDB ATOM/HETATM; tempFactor (B-factor) in cols 61-66 -> slice [60:66]
                if line.startswith(("ATOM  ", "HETATM")) and len(line) >= 66:
                    try:
                        b = float(line[60:66])
                        plddts.append(b)
                    except ValueError:
                        continue
    except OSError as e:
        warn(f"Could not read {filepath}: {e}")
        return None

    if not plddts:
        warn(f"No pLDDT (B-factor) values found in {os.path.basename(filepath)}.")
        return None

    a = np.array(plddts, dtype=float)
    return {
        "n_atoms": int(a.size),
        "plddt_mean": float(a.mean()),
        "plddt_median": float(np.median(a)),
        "pct_ge_90": float((a >= 90).mean() * 100.0),
        "pct_ge_70": float((a >= 70).mean() * 100.0),
        "pct_lt_50": float((a < 50).mean() * 100.0),
    }

# ---- Main loop to download structures, apply threshold, and collect summary ----
summary_rows = []

for _, row in df.iterrows():
    accession = row[f"{id_column_name}"]
    pdb_file = download_pdb(accession, pdb_dir)

    row_summary = {
        "accession": accession,
        "downloaded": False,
        "kept": False,
        "reason": "",
        "n_atoms": np.nan,
        "plddt_mean": np.nan,
        "plddt_median": np.nan,
        "pct_ge_90": np.nan,
        "pct_ge_70": np.nan,
        "pct_lt_50": np.nan,
        "pdb_path": None,
    }

    if pdb_file is None:
        warn(f"Skipping {accession}: no file downloaded.")
        row_summary["reason"] = "download_failed"
        summary_rows.append(row_summary)
        continue

    row_summary["downloaded"] = True
    row_summary["pdb_path"] = pdb_file

    stats = plddt_stats_from_pdb(pdb_file)
    if stats is None:
        warn(f"{accession}: could not compute pLDDT stats; deleting file.")
        try:
            os.remove(pdb_file)
            row_summary["reason"] = "no_plddt_deleted"
        except OSError as e:
            warn(f"{accession}: failed to delete corrupt file: {e}")
            row_summary["reason"] = "no_plddt_delete_failed"
        summary_rows.append(row_summary)
        continue

    # fill stats
    for k, v in stats.items():
        row_summary[k] = v

    print(f"{accession}: pLDDT mean={stats['plddt_mean']:.1f}, median={stats['plddt_median']:.1f}")

    # thresholding & optional deletion
    if stats["plddt_mean"] < PLDDT_KEEP_MIN:
        msg = (f"{accession}: mean pLDDT {stats['plddt_mean']:.1f} < "
               f"threshold {PLDDT_KEEP_MIN:.1f}")
        if DELETE_LOW_CONF:
            warn(msg + " — deleting file.")
            try:
                os.remove(pdb_file)
                row_summary["reason"] = "below_threshold_deleted"
            except OSError as e:
                warn(f"{accession}: failed to delete low-confidence file: {e}")
                row_summary["reason"] = "below_threshold_delete_failed"
        else:
            warn(msg + " — keeping file (DELETE_LOW_CONF=False).")
            row_summary["reason"] = "below_threshold_kept"
            row_summary["kept"] = True
    else:
        row_summary["kept"] = True
        row_summary["reason"] = "ok"

    summary_rows.append(row_summary)

# ---- Build summary DataFrame and save to CSV ----
summary_df = pd.DataFrame(summary_rows)
csv_path = os.path.join(pdb_dir, SUMMARY_CSV)
summary_df.to_csv(csv_path, index=False)
info(f"Saved summary to {csv_path}")
print("Download process complete.")

[INFO] Downloaded A0A1A5PGB3.pdb
A0A1A5PGB3: pLDDT mean=96.5, median=98.2
[INFO] Downloaded A0A1Q4V950.pdb
A0A1Q4V950: pLDDT mean=94.5, median=98.1
[INFO] Downloaded A0A841F4E4.pdb
A0A841F4E4: pLDDT mean=88.1, median=91.6
[INFO] Downloaded A0A003.pdb
A0A003: pLDDT mean=89.6, median=96.1
[INFO] Downloaded A0A004.pdb
A0A004: pLDDT mean=81.5, median=88.3
[INFO] Downloaded A0A1R0M6B7.pdb
A0A1R0M6B7: pLDDT mean=86.7, median=94.5


[WARN] A0A5J6G3N3: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A5J6G3N3: no file downloaded.
[WARN] A0A009DWE1: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A009DWE1: no file downloaded.
[WARN] A0A009DWN1: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A009DWN1: no file downloaded.
[WARN] A0A009DWW6: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A009DWW6: no file downloaded.
[WARN] A0A841NBK8: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A841NBK8: no file downloaded.


[INFO] Downloaded A0A0A8TFF6.pdb
A0A0A8TFF6: pLDDT mean=87.2, median=90.8


[WARN] A0A009DXY7: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A009DXY7: no file downloaded.
[WARN] A0A009DZ65: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A009DZ65: no file downloaded.


[INFO] Downloaded N9MM00.pdb
N9MM00: pLDDT mean=89.3, median=91.8


...

[INFO] Downloaded P49126.pdb
P49126: pLDDT mean=79.0, median=84.7
[INFO] Downloaded P61100.pdb
P61100: pLDDT mean=76.5, median=77.7
[INFO] Downloaded P81511.pdb
P81511: pLDDT mean=84.1, median=90.9
[INFO] Downloaded P85511.pdb
P85511: pLDDT mean=89.9, median=93.6
[INFO] Downloaded Q04187.pdb
Q04187: pLDDT mean=75.2, median=90.5
[INFO] Downloaded Q04188.pdb
Q04188: pLDDT mean=60.8, median=63.4
[INFO] Downloaded Q1A2D2.pdb
Q1A2D2: pLDDT mean=88.6, median=90.1


[WARN] Q04188: mean pLDDT 60.8 < threshold 70.0 — deleting file.


[INFO] Downloaded Q47182.pdb
Q47182: pLDDT mean=86.0, median=91.1
[INFO] Downloaded Q892G2.pdb
Q892G2: pLDDT mean=94.1, median=97.1
[INFO] Downloaded Q90603.pdb
Q90603: pLDDT mean=77.7, median=89.4
[INFO] Downloaded Q96TL7.pdb
Q96TL7: pLDDT mean=67.8, median=56.2
[INFO] Downloaded A5JUR2.pdb
A5JUR2: pLDDT mean=64.1, median=66.2


[WARN] Q96TL7: mean pLDDT 67.8 < threshold 70.0 — deleting file.
[WARN] A5JUR2: mean pLDDT 64.1 < threshold 70.0 — deleting file.
[WARN] A0A3D9VT81: AFDB entry not found (HTTP 404).
[WARN] Skipping A0A3D9VT81: no file downloaded.
[WARN] T0M3B2: AFDB entry not found (HTTP 404).
[WARN] Skipping T0M3B2: no file downloaded.


[INFO] Downloaded W2CDM8.pdb
W2CDM8: pLDDT mean=93.8, median=95.1
[INFO] Downloaded X1JWS5.pdb
X1JWS5: pLDDT mean=83.0, median=89.4
[INFO] Saved summary to ted_structures/ted_structures_download_summary.csv
Download process complete.


In [17]:
summary_df.query("kept == True")

,accession,downloaded,kept,reason,n_atoms,plddt_mean,plddt_median,pct_ge_90,pct_ge_70,pct_lt_50,pdb_path
0,A0A1A5PGB3,True,True,ok,2992.0,96.458446,98.19,94.184492,98.663102,0.000000,ted_structures/A0A1A5PGB3.pdb
1,A0A1Q4V950,True,True,ok,3141.0,94.495982,98.06,88.061127,95.702006,2.387775,ted_structures/A0A1Q4V950.pdb
2,A0A841F4E4,True,True,ok,4694.0,88.127635,91.56,58.798466,92.948445,2.705582,ted_structures/A0A841F4E4.pdb
3,A0A003,True,True,ok,2536.0,89.570631,96.12,78.667192,89.116719,6.703470,ted_structures/A0A003.pdb
4,A0A004,True,True,ok,4890.0,81.523047,88.31,43.292434,81.083845,9.366053,ted_structures/A0A004.pdb
...,...,...,...,...,...,...,...,...,...,...,...
5390,Q47182,True,True,ok,569.0,86.021986,91.06,56.239016,87.873462,0.000000,ted_structures/Q47182.pdb
5391,Q892G2,True,True,ok,1062.0,94.146309,97.12,84.651601,97.269303,0.000000,ted_structures/Q892G2.pdb
5392,Q90603,True,True,ok,3423.0,77.748875,89.38,47.735904,70.873503,21.267894,ted_structures/Q90603.pdb
5397,W2CDM8,True,True,ok,2828.0,93.830629,95.12,85.749646,99.222065,0.000000,ted_structures/W2CDM8.pdb
